In [ ]:
import sys
sys.path.insert(0, "..")

# Data access

In [ ]:
from src.data import *

## Direct table access

Each source (identified by `source_id`) is published as a single CSV file for each week. The CSVs contain the **free dates** that have been reported by the website at each snapshot time (typically every 15 minutes).

The tables can be accessed with the `Data.get_table` class method:

In [ ]:
columns, rows = Data.get_table((2021, 30), "leipzig")
print(columns[:5])
print(rows[0][:5])

Or with the `Data.get_dataframe` class method which returns a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html):

In [ ]:
Data.get_dataframe((2021, 30), "leipzig")

For convenience, both methods support adding the names from the [metadata.json](compressed/metadata.json) file:

In [ ]:
Data.get_dataframe((2021, 30), "leipzig", with_meta=True)

Dataframes are very useful to analyze the data. For example, we can count the sum of all free dates available for each sub-location:

In [ ]:
df = Data.get_dataframe((2021, 30), "leipzig", with_meta=True)
loc_sum = df.groupby("location_name").sum().sum(axis=1).sort_values()
loc_sum.plot.barh(figsize=(10, 6))

### Note on filters

All string filters (for `source_id`, `location_id` or [metric name](#precalculated-metrics-access) support:

- wildcard matching. e.g. `"bonn*"` matches `"bonn"` and `"bonnbau"`
- lists of wildcards. e.g. `["bonn*", "dresden*"]`
- functions. e.g. `lambda n: n.startswith("bonn")`

## Iterate through all tables

To access all tables in the published dataset you can use the `Data.iter_*` methods. The `Data` class has to be instantiated and will accept *filters* in the constructor:

In [ ]:
data = Data(source_id=["jena", "bonn*"], iso_week_lt=(2021, 30))

for week, source_id, df in data.iter_dataframes():
    print(f"{week} {source_id:10s} shape={df.shape}")

The returned tables or dataframes are always sorted first by week and secondary by source_id. From these tables, it's possible to calculate all kinds of metrics, although it takes some time as the uncompressed tables are gigabytes in size altogether. 

For the impatient there are some precalcuated metrics contained in this repository.

# Precalculated metrics access

The available metrics are `free_dates`, `appointments` and `cancellations`. The table rows are the snapshot dates (truncated to exact 15 minutes steps) and the columns contain each metric for each source and sub-location. The column names are concatenated strings of `source_id`/`location_id`/`metric_name`.

The above example of free dates per sub-location can be repeated using:

In [ ]:
df = Metrics.dataframe("free_dates", "leipzig", iso_week=(2021, 30))

which gives us the filtered metrics:

In [ ]:
df

which can likewise be summed and plotted:

In [ ]:
df.sum().sort_values().plot.barh(figsize=(10, 6))

### Appointments and cancellations

The number of appointments that have been made between two website snapshots, as well as the number of cancelled appointments are estimated from the raw data. 

For example we can plot these metrics for a all locations of a source, summed for each day:

In [ ]:
df = Metrics.dataframe(
    ["appointments", "cancellations"], "blankenburg",
    iso_week_gte=(2021, 30), iso_week_lte=(2021, 37),  # put a time range to the returned data
)
df.resample("1d").sum().plot(figsize=(16, 4))

### Metrics with timespans

All of the metrics are also available according to their distance to the snapshot time. For example, `appointments_0d` holds the number of appointments made at the same day as the snapshot, or `free_dates_1h` holds the number of free dates that are 1 hour apart from the snapshot date. Snapshot and possible appointment dates are quantized
to full hours, days or calendar weeks before calculating the distance.

Using wildcard matching, we can retrieve all timespans for a certain metric: 

In [ ]:
df = Metrics.dataframe(
    ["appointments_*w"], "blankenburg", "85150", 
    iso_week_gte=(2021, 30), iso_week_lte=(2021, 37),
)
df.resample("1d").sum().plot(figsize=(16, 4))

We can see that appointments made in the same week (`appointments_0w`) are quite rare, and appointments within the next week (`appointments_1w`) are getting rarer while appointments made in the week after next week (`appointments_2w`) are growing.

If we compare with the number of free dates available within each weekly timespan, it kind of makes sense and probably is a result of people coming home from holidays and getting busy again:

In [ ]:
df = Metrics.dataframe(
    ["free_dates_*w"], "blankenburg", "85150",
    iso_week_gte=(2021, 30), iso_week_lte=(2021, 37),
)
df.resample("1d").sum().plot(figsize=(16, 4))